In [1]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [2]:
import os
os.environ["HF_DATASETS_CACHE"] = "/shared/3/projects/bangzhao/.hf_cache"
from datasets import load_dataset

In [3]:
import torch
from torch.utils.data import Dataset, DataLoader
import random
from torch.optim import AdamW
import torch.nn as nn

from transformers import *
from tokenizers import *

from bertPhoneme import BertEmbeddingsV2, BertModelV2, BertForMaskedLMV2, BertConfigV2, MaskedLMWithProsodyOutput

2025-04-21 15:32:43.988574: I tensorflow/core/platform/cpu_feature_guard.cc:181] Beginning TensorFlow 2.15, this package will be updated to install stock TensorFlow 2.15 alongside Intel's TensorFlow CPU extension plugin, which provides all the optimizations available in the package and more. If a compatible version of stock TensorFlow is present, only the extension will get installed. No changes to code or installation setup is needed as a result of this change.
More information on Intel's optimizations for TensorFlow, delivered as TensorFlow extension plugin can be viewed at https://github.com/intel/intel-extension-for-tensorflow.
2025-04-21 15:32:43.988644: I tensorflow/core/platform/cpu_feature_guard.cc:192] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


[2025-04-21 15:32:46,200] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/opt/anaconda/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/opt/anaconda/compiler_compat/ld: warning: librt.so.1, needed by /usr/local/cuda-12.4/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/opt/anaconda/compiler_compat/ld: warning: libpthread.so.0, needed by /usr/local/cuda-12.4/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/opt/anaconda/compiler_compat/ld: warning: libstdc++.so.6, needed by /usr/local/cuda-12.4/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/opt/anaconda/compiler_compat/ld: warning: libm.so.6, needed by /usr/local/cuda-12.4/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/opt/anaconda/compiler_compat/ld: /usr/local/cuda-12.4/lib64/libcufile.so: undefined reference to `std::runtime_error::~runtime_error()@GLIBCXX_3.4'
/opt/anaconda/compiler_compat/ld: /usr/local/cuda-12.4/lib64/libcufile.so: undefined reference to `__gxx_personality_v0@C

In [4]:
import json
import random

In [5]:
file_path = "/shared/3/projects/bangzhao/prosodic_embeddings/merge/training_data/output_part_1.jsonl"
phonemes = set()

with open(file_path, 'r') as f:
    for i, line in enumerate(f):
        if i >= 5000:
            break
        phonemes.update(set(json.loads(line)['phoneme']))
    print(len(phonemes))

70


In [6]:
phonemes.update(['UNK'])

In [7]:
phoneme_vocab = {p: i for i, p in enumerate(phonemes)}

In [8]:
print(phoneme_vocab)

{'OW1': 0, 'R': 1, 'AE2': 2, 'TH': 3, 'CH': 4, 'IH0': 5, 'EH2': 6, 'ER2': 7, 'AA2': 8, 'D': 9, 'K': 10, 'OW0': 11, 'AY2': 12, 'Z': 13, 'UH2': 14, 'IH1': 15, 'AA0': 16, 'W': 17, 'SH': 18, 'IY2': 19, 'F': 20, 'AO1': 21, 'UH1': 22, 'OW2': 23, 'EH1': 24, 'AW1': 25, 'AY0': 26, 'AH1': 27, 'OY1': 28, 'spn': 29, 'OY0': 30, 'EY0': 31, 'ER1': 32, 'IY1': 33, 'IH2': 34, 'V': 35, 'AY1': 36, 'AO0': 37, 'ZH': 38, 'NG': 39, 'AE1': 40, 'AO2': 41, 'UW0': 42, 'AH0': 43, 'UW1': 44, 'AW2': 45, 'Y': 46, 'ER0': 47, 'IY0': 48, 'AH2': 49, 'UW2': 50, 'EY1': 51, 'S': 52, 'G': 53, 'AE0': 54, 'JH': 55, 'DH': 56, 'L': 57, 'P': 58, 'AA1': 59, 'OY2': 60, 'UH0': 61, 'B': 62, 'UNK': 63, 'EY2': 64, 'M': 65, 'EH0': 66, 'T': 67, 'AW0': 68, 'HH': 69, 'N': 70}


In [9]:
with open("/shared/3/projects/bangzhao/prosodic_embeddings/bert_train/phoneme_vocab_prosody.json", "w") as f:
    json.dump(phoneme_vocab, f, indent=2)

In [5]:
with open("/shared/3/projects/bangzhao/prosodic_embeddings/bert_train/phoneme_vocab_prosody.json", "r") as f:
    phoneme_vocab = json.load(f)

In [6]:
phoneme_vocab_size = len(phoneme_vocab)
pad_token_id = 71
pad_cluster_id = 201
mask_prosody_id = 200

class HuggingFacePhonemeDataset(Dataset):
    def __init__(self, hf_dataset, vocab, mask_prob=0.15, max_length=512):
        self.dataset = hf_dataset
        self.vocab = vocab
        self.mask_prob = mask_prob
        self.max_length = max_length

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        sample = self.dataset[idx]

        phonemes = sample["phoneme"]
        prosody_ids = sample["prosody_id_200"]

        # Tokenize and truncate
        input_ids = [self.vocab.get(p, self.vocab["UNK"]) for p in phonemes][:self.max_length]
        prosody_ids = prosody_ids[:self.max_length]

        # Create prosody_labels and apply masking
        prosody_labels = prosody_ids.copy()
        for i in range(len(prosody_ids)):
            if random.random() < self.mask_prob:
                prosody_labels[i] = prosody_ids[i]       # Keep true label for loss
                prosody_ids[i] = mask_prosody_id         # Mask token
            else:
                prosody_labels[i] = -100                 # Ignore in loss

        # Padding
        pad_length = self.max_length - len(input_ids)
        input_ids.extend([pad_token_id] * pad_length)
        prosody_ids.extend([pad_cluster_id] * pad_length)
        prosody_labels.extend([-100] * pad_length)
        attention_mask = [1] * (self.max_length - pad_length) + [0] * pad_length

        return {
            "input_ids": torch.tensor(input_ids, dtype=torch.long),
            "prosody_ids": torch.tensor(prosody_ids, dtype=torch.long),
            "prosody_labels": torch.tensor(prosody_labels, dtype=torch.long),
            "attention_mask": torch.tensor(attention_mask, dtype=torch.long),
        }

In [7]:
hf_dataset = load_dataset("json", data_files="/shared/3/projects/bangzhao/prosodic_embeddings/merge/training_data/output_part_1.jsonl", split="train")
hf_dataset = hf_dataset.select(range(49000))


Loading dataset shards:   0%|          | 0/60 [00:00<?, ?it/s]

In [8]:
len(hf_dataset)

49000

In [9]:
train_dataset = HuggingFacePhonemeDataset(hf_dataset, phoneme_vocab)

In [11]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [14]:
vocab_size = len(phoneme_vocab.keys())
phoneme_vocab_size = len(phoneme_vocab)
max_length = 512


model_config = BertConfigV2(
    vocab_size=vocab_size+1,
    pad_token_id=pad_token_id,
    pad_cluster_id=201,
    hidden_size=128,
    num_hidden_layers=2,
    num_attention_heads=2,
    intermediate_size=512,
    max_position_embeddings=max_length,
    prosody_cluster_size=200 + 2
)

model = BertForMaskedLMV2(config=model_config)

# BERT-Base	768	12	12	3072
# BERT-Small 512	4	8	2048
# BERT-Mini	256	4	4	1024
# BERT-Tiny	128	2	2	512

In [15]:
model

BertForMaskedLMV2(
  (bert): BertModelV2(
    (embeddings): BertEmbeddingsV2(
      (word_embeddings): Embedding(72, 128, padding_idx=71)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (prosody_embeddings): Embedding(202, 128, padding_idx=201)
      (conv): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = AdamW(model.parameters(), lr=5e-4)

# mlm_loss_fn = nn.CrossEntropyLoss()
prosody_loss_fn = nn.CrossEntropyLoss(ignore_index=-100)

num_epochs = 3
model.train()

for epoch in range(num_epochs):
    total_loss = 0

    print(f"\n Epoch {epoch + 1} starting...\n")

    for step, batch in enumerate(train_loader):
        input_ids = batch["input_ids"].to(device)
        prosody_ids = batch["prosody_ids"].to(device)
        labels = batch["prosody_labels"].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, prosody_ids=prosody_ids)

        # mlm_loss = mlm_loss_fn(outputs.logits.view(-1, model.config.vocab_size), labels.view(-1))
        prosody_loss = prosody_loss_fn(outputs.prosody_logits.view(-1, model.config.prosody_cluster_size), labels.view(-1))

        total_batch_loss = prosody_loss # + mlm_loss
        total_batch_loss.backward()
        optimizer.step()

        total_loss += total_batch_loss.item()

        # Report every 100 steps
        if (step + 1) % 100 == 0:
            avg_loss = total_loss / (step + 1)
            print(f"  Step {step + 1:>4} | Avg Loss: {avg_loss:.4f}")

    avg_epoch_loss = total_loss / len(train_loader)
    print(f"\n Epoch {epoch + 1} completed. Avg Loss: {avg_epoch_loss:.4f}")



 Epoch 1 starting...



/opt/anaconda/lib/python3.11/site-packages/transformers/modeling_utils.py:1575: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


  Step  100 | Avg Loss: 4.7256
  Step  200 | Avg Loss: 4.5847
  Step  300 | Avg Loss: 4.4913
  Step  400 | Avg Loss: 4.4346
  Step  500 | Avg Loss: 4.3942
  Step  600 | Avg Loss: 4.3649
  Step  700 | Avg Loss: 4.3422
  Step  800 | Avg Loss: 4.3239
  Step  900 | Avg Loss: 4.3086
  Step 1000 | Avg Loss: 4.2957
  Step 1100 | Avg Loss: 4.2846
  Step 1200 | Avg Loss: 4.2743
  Step 1300 | Avg Loss: 4.2657
  Step 1400 | Avg Loss: 4.2585
  Step 1500 | Avg Loss: 4.2516

 Epoch 1 completed. Avg Loss: 4.2494

 Epoch 2 starting...

  Step  100 | Avg Loss: 4.1396
  Step  200 | Avg Loss: 4.1452
  Step  300 | Avg Loss: 4.1420
  Step  400 | Avg Loss: 4.1412
  Step  500 | Avg Loss: 4.1401
  Step  600 | Avg Loss: 4.1388
  Step  700 | Avg Loss: 4.1376
  Step  800 | Avg Loss: 4.1376
  Step  900 | Avg Loss: 4.1358
  Step 1000 | Avg Loss: 4.1343
  Step 1100 | Avg Loss: 4.1341
  Step 1200 | Avg Loss: 4.1331
  Step 1300 | Avg Loss: 4.1320
  Step 1400 | Avg Loss: 4.1306
  Step 1500 | Avg Loss: 4.1298

 Epoch 2

In [17]:
hf_dataset_full = load_dataset(
    "json",
    data_files="/shared/3/projects/bangzhao/prosodic_embeddings/merge/training_data/output_part_1.jsonl",
    split="train"
)
hf_test_dataset = hf_dataset_full.select(range(49000, 50000))

# === Step 2: Wrap the dataset with HuggingFacePhonemeDataset ===
test_dataset = HuggingFacePhonemeDataset(
    hf_dataset=hf_test_dataset,
    vocab=phoneme_vocab,
    mask_prob=0.15,
    max_length=512
)

Loading dataset shards:   0%|          | 0/60 [00:00<?, ?it/s]

In [27]:
sample = test_dataset[3]

input_ids = sample["input_ids"].unsqueeze(0).to(model.device)
labels = sample["prosody_labels"].unsqueeze(0).to(model.device)
prosody_ids = sample["prosody_ids"].unsqueeze(0).to(model.device)
attention_mask = sample["attention_mask"].unsqueeze(0).to(model.device)

In [29]:
model.eval()
with torch.no_grad():
    outputs = model(input_ids=input_ids, prosody_ids=prosody_ids, attention_mask=attention_mask)

pred_phonemes = torch.argmax(outputs.logits, dim=-1)[0]         # [seq_len]
pred_prosody = torch.argmax(outputs.prosody_logits, dim=-1)[0]  # [seq_len]

# Evaluation
id2phoneme = {v: k for k, v in phoneme_vocab.items()}
mask_positions = (labels != -100).nonzero(as_tuple=True)[1].tolist()

results = []
correct_phoneme = 0
correct_prosody = 0

for i in mask_positions:
    gt_ph_id = labels[0, i].item()
    pred_ph_id = pred_phonemes[i].item()

    gt_pr_id = labels[0, i].item()
    pred_pr_id = pred_prosody[i].item()

    phoneme_correct = gt_ph_id == pred_ph_id
    prosody_correct = gt_pr_id == pred_pr_id

    if phoneme_correct:
        correct_phoneme += 1
    if prosody_correct:
        correct_prosody += 1

    results.append({
        "position": i,
        "phoneme_correct": phoneme_correct,
        "phoneme_gt": id2phoneme.get(gt_ph_id, "UNK"),
        "phoneme_pred": id2phoneme.get(pred_ph_id, "UNK"),
        "phoneme_gt_id": gt_ph_id,
        "phoneme_pred_id": pred_ph_id,
        "prosody_correct": prosody_correct,
        "prosody_gt_id": gt_pr_id,
        "prosody_pred_id": pred_pr_id,
    })

# Accuracy
total = len(mask_positions)
phoneme_accuracy = correct_phoneme / total if total > 0 else None
prosody_accuracy = correct_prosody / total if total > 0 else None

#print(f"\nPhoneme Accuracy: {phoneme_accuracy:.2%}" if phoneme_accuracy is not None else "No masked phoneme positions.")
print(f"Prosody Accuracy: {prosody_accuracy:.2%}" if prosody_accuracy is not None else "No masked prosody positions.")

for r in results:
    # print(f"Pos {r['position']:>3} | Phoneme: Pred={r['phoneme_pred']} GT={r['phoneme_gt']} → {'True' if r['phoneme_correct'] else 'False'}")
    print(f" | Prosody: Pred={r['prosody_pred_id']} GT={r['prosody_gt_id']} → {'True' if r['prosody_correct'] else 'False'}")


Prosody Accuracy: 9.68%
 | Prosody: Pred=7 GT=69 → False
 | Prosody: Pred=69 GT=56 → False
 | Prosody: Pred=7 GT=8 → False
 | Prosody: Pred=7 GT=98 → False
 | Prosody: Pred=153 GT=138 → False
 | Prosody: Pred=7 GT=158 → False
 | Prosody: Pred=7 GT=56 → False
 | Prosody: Pred=153 GT=187 → False
 | Prosody: Pred=153 GT=179 → False
 | Prosody: Pred=7 GT=76 → False
 | Prosody: Pred=153 GT=195 → False
 | Prosody: Pred=153 GT=97 → False
 | Prosody: Pred=153 GT=131 → False
 | Prosody: Pred=7 GT=195 → False
 | Prosody: Pred=153 GT=161 → False
 | Prosody: Pred=153 GT=21 → False
 | Prosody: Pred=153 GT=123 → False
 | Prosody: Pred=7 GT=131 → False
 | Prosody: Pred=173 GT=191 → False
 | Prosody: Pred=153 GT=76 → False
 | Prosody: Pred=153 GT=71 → False
 | Prosody: Pred=7 GT=176 → False
 | Prosody: Pred=153 GT=91 → False
 | Prosody: Pred=7 GT=140 → False
 | Prosody: Pred=131 GT=158 → False
 | Prosody: Pred=153 GT=153 → True
 | Prosody: Pred=7 GT=56 → False
 | Prosody: Pred=71 GT=153 → False
 | Pro

In [30]:
from tqdm import tqdm  # Optional: for progress bar

total_phoneme_correct = 0
total_prosody_correct = 0
total_masked_positions = 0

model.eval()

for idx in tqdm(range(len(test_dataset)), desc="Evaluating samples"):
    sample = test_dataset[idx]

    input_ids = sample["input_ids"].unsqueeze(0).to(model.device)
    labels = sample["prosody_labels"].unsqueeze(0).to(model.device)
    prosody_ids = sample["prosody_ids"].unsqueeze(0).to(model.device)
    attention_mask = sample["attention_mask"].unsqueeze(0).to(model.device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, prosody_ids=prosody_ids, attention_mask=attention_mask)

    pred_phonemes = torch.argmax(outputs.logits, dim=-1)[0]
    pred_prosody = torch.argmax(outputs.prosody_logits, dim=-1)[0]
    mask_positions = (labels != -100).nonzero(as_tuple=True)[1].tolist()

    for i in mask_positions:
        gt_ph_id = labels[0, i].item()
        pred_ph_id = pred_phonemes[i].item()
        gt_pr_id = labels[0, i].item()
        pred_pr_id = pred_prosody[i].item()

        total_masked_positions += 1
        if gt_ph_id == pred_ph_id:
            total_phoneme_correct += 1
        if gt_pr_id == pred_pr_id:
            total_prosody_correct += 1

# === Final accuracy calculation ===
if total_masked_positions > 0:
    avg_phoneme_acc = total_phoneme_correct / total_masked_positions
    avg_prosody_acc = total_prosody_correct / total_masked_positions
else:
    avg_phoneme_acc = avg_prosody_acc = None

print(f"\n✅ Evaluated {total_masked_positions} masked positions over {len(test_dataset)} samples")
# print(f"Phoneme Accuracy: {avg_phoneme_acc:.2%}" if avg_phoneme_acc is not None else "No masked phoneme positions.")
print(f"Prosody Accuracy: {avg_prosody_acc:.2%}" if avg_prosody_acc is not None else "No masked prosody positions.")

Evaluating samples: 100%|████████████████████| 1000/1000 [00:22<00:00, 43.71it/s]


✅ Evaluated 74316 masked positions over 1000 samples
Prosody Accuracy: 11.93%
